In [1]:
import pyspark
from pyspark.sql import SparkSession
from itertools import islice
from pyspark.sql import Row
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder.appName('pyspark_training')\
                            .master('local[10]').getOrCreate()

# - Create Dataframe using list

In [3]:
row_list = [('John',26,'Pune'),('Jessica',25,'NY'),('Robert',24,'LA'),('Adam',28,'California')]

In [4]:
col_names = ['Name','Age','City']

In [5]:
df_from_list = spark.createDataFrame(row_list,col_names)
df_from_list.show()

+-------+---+----------+
|   Name|Age|      City|
+-------+---+----------+
|   John| 26|      Pune|
|Jessica| 25|        NY|
| Robert| 24|        LA|
|   Adam| 28|California|
+-------+---+----------+



# - Create DataFrame using dict - depricated

In [6]:
row_dict = [{'Name':'John','Age':26},{'Name':'Jessica','Age':25},
            {'Name':'Robert','Age':24},{'Name':'Adam','Age':28}]

In [7]:
df_from_dict = spark.createDataFrame(row_dict)

In [8]:
df_from_dict.show()

+---+-------+
|Age|   Name|
+---+-------+
| 26|   John|
| 25|Jessica|
| 24| Robert|
| 28|   Adam|
+---+-------+



# - Create DataFrame using RDD

In [9]:
Person = Row('Name','Age','City')

In [10]:
Person

<Row('Name', 'Age', 'City')>

In [11]:
rdd = spark.sparkContext.parallelize(row_list)

In [12]:
rdd.collect()

[('John', 26, 'Pune'),
 ('Jessica', 25, 'NY'),
 ('Robert', 24, 'LA'),
 ('Adam', 28, 'California')]

In [13]:
person = rdd.map(lambda r:Person(*r))

In [14]:
person.collect()

[Row(Name='John', Age=26, City='Pune'),
 Row(Name='Jessica', Age=25, City='NY'),
 Row(Name='Robert', Age=24, City='LA'),
 Row(Name='Adam', Age=28, City='California')]

In [15]:
df_from_rdd = spark.createDataFrame(person)

In [16]:
df_from_rdd.show()

+-------+---+----------+
|   Name|Age|      City|
+-------+---+----------+
|   John| 26|      Pune|
|Jessica| 25|        NY|
| Robert| 24|        LA|
|   Adam| 28|California|
+-------+---+----------+



# - Create DataFrame by RDD created by reading a textfile

In [17]:
rdd_us_cars = spark.sparkContext.textFile('USA_cars_datasets.csv')

In [18]:
rdd_us_cars.take(2)

['row_id,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition',
 '0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,  jtezu11f88k007763,159348797,new jersey, usa,10 days left']

In [19]:
header = rdd_us_cars.first()
rdd_us_cars = rdd_us_cars.filter(lambda line:line!=header)

In [20]:
rdd_us_cars.take(5)

['0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,  jtezu11f88k007763,159348797,new jersey, usa,10 days left',
 '1,2899,ford,se,2011,clean vehicle,190552.0,silver,  2fmdk3gc4bbb02217,166951262,tennessee, usa,6 days left',
 '2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,  3c4pdcgg5jt346413,167655728,georgia, usa,2 days left',
 '3,25000,ford,door,2014,clean vehicle,64146.0,blue,  1ftfw1et4efc23745,167753855,virginia, usa,22 hours left',
 '4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,  3gcpcrec2jg473991,167763266,florida, usa,22 hours left']

In [21]:
rdd_us_cars_mapped = rdd_us_cars.map(lambda r:r.split(','))

In [22]:
rdd_us_cars_mapped.take(1)

[['0',
  '6300',
  'toyota',
  'cruiser',
  '2008',
  'clean vehicle',
  '274117.0',
  'black',
  '  jtezu11f88k007763',
  '159348797',
  'new jersey',
  ' usa',
  '10 days left']]

In [23]:
cols = ['row_id','price','brand','model','year','title_status','mileage','color','vin','lot','state','country','condition']

In [24]:
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType

In [25]:
schema = StructType([StructField(col,StringType(),True) for col in cols])

In [26]:
schema

StructType(List(StructField(row_id,StringType,true),StructField(price,StringType,true),StructField(brand,StringType,true),StructField(model,StringType,true),StructField(year,StringType,true),StructField(title_status,StringType,true),StructField(mileage,StringType,true),StructField(color,StringType,true),StructField(vin,StringType,true),StructField(lot,StringType,true),StructField(state,StringType,true),StructField(country,StringType,true),StructField(condition,StringType,true)))

In [27]:
df_us_cars = spark.createDataFrame(rdd_us_cars_mapped,schema)

In [28]:
df_us_cars.show()

+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|row_id|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|
+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|     0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|
|     1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|
|     2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|
|     3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|
|     4|27700|chevrolet|   1500|2018|clean vehicle|  66

# - Read a file using a csv api

In [29]:
df_us_cars = spark.read.option('header','true').csv('USA_cars_datasets.csv')

In [30]:
df_us_cars.show()

+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|row_id|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|
+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|     0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|
|     1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|
|     2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|
|     3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|
|     4|27700|chevrolet|   1500|2018|clean vehicle|  66

In [31]:
df_us_cars.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- year: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- color: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- lot: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- condition: string (nullable = true)



# - Get the dataframe schema

In [32]:
schema1 = df_us_cars.schema

In [33]:
schema1

StructType(List(StructField(row_id,StringType,true),StructField(price,StringType,true),StructField(brand,StringType,true),StructField(model,StringType,true),StructField(year,StringType,true),StructField(title_status,StringType,true),StructField(mileage,StringType,true),StructField(color,StringType,true),StructField(vin,StringType,true),StructField(lot,StringType,true),StructField(state,StringType,true),StructField(country,StringType,true),StructField(condition,StringType,true)))

# - Get the dataframe columns

In [34]:
cars_cols = df_us_cars.columns

In [35]:
cars_cols

['row_id',
 'price',
 'brand',
 'model',
 'year',
 'title_status',
 'mileage',
 'color',
 'vin',
 'lot',
 'state',
 'country',
 'condition']

# first row

In [36]:
df_us_cars.first()

Row(row_id='0', price='6300', brand='toyota', model='cruiser', year='2008', title_status='clean vehicle', mileage='274117.0', color='black', vin='  jtezu11f88k007763', lot='159348797', state='new jersey', country=' usa', condition='10 days left')

# get n rows

In [37]:
df_us_cars.take(10)

[Row(row_id='0', price='6300', brand='toyota', model='cruiser', year='2008', title_status='clean vehicle', mileage='274117.0', color='black', vin='  jtezu11f88k007763', lot='159348797', state='new jersey', country=' usa', condition='10 days left'),
 Row(row_id='1', price='2899', brand='ford', model='se', year='2011', title_status='clean vehicle', mileage='190552.0', color='silver', vin='  2fmdk3gc4bbb02217', lot='166951262', state='tennessee', country=' usa', condition='6 days left'),
 Row(row_id='2', price='5350', brand='dodge', model='mpv', year='2018', title_status='clean vehicle', mileage='39590.0', color='silver', vin='  3c4pdcgg5jt346413', lot='167655728', state='georgia', country=' usa', condition='2 days left'),
 Row(row_id='3', price='25000', brand='ford', model='door', year='2014', title_status='clean vehicle', mileage='64146.0', color='blue', vin='  1ftfw1et4efc23745', lot='167753855', state='virginia', country=' usa', condition='22 hours left'),
 Row(row_id='4', price='2770

In [38]:
df_us_cars

DataFrame[row_id: string, price: string, brand: string, model: string, year: string, title_status: string, mileage: string, color: string, vin: string, lot: string, state: string, country: string, condition: string]

# Preparing the Loan Dataset

In [39]:
df_loans = spark.read.parquet('loan_data.parquet')

In [40]:
df_loans.columns

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model']

In [41]:
df_loans.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_name: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- description: string (nullable = true)
 |-- description_translated: string (nullable = true)
 |-- funded_amount: double (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- status: string (nullable = true)
 |-- activity_name: string (nullable = true)
 |-- sector_name: string (nullable = true)
 |-- loan_use: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- town_name: string (nullable = true)
 |-- currency_policy: string (nullable = true)
 |-- currency_exchange_coverage_rate: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- partner_id: double (nullable = true)
 |-- posted_time: string (nullable = true)
 |-- planned_expiration_time: string (nullable = true)
 |-- disburse_time: string (nullable = true)
 |-- raised_time: string (nullable = true)
 |-- len

In [42]:
df_loans.show(10,False)

+-------+----------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# - Get first row

In [43]:
x = df_loans.first()

In [44]:
x

Row(loan_id='1057058', loan_name='\\tLucy', original_language='English', description='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish selling business for over four years, selling from home to restaurant operators, and passersby. She faces a major challenge of seasonality and perishability leading to losses.<br /><br />With the KES 30,000, Lucy wants to purchase bundles of fish for resale. She dreams of expanding her business and establishing another business in the near future.', description_translated='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish se

# - Different Loan Status

In [45]:
df_loans.select('status').distinct().show()

+-----------+
|     status|
+-----------+
|   refunded|
|fundRaising|
|    expired|
|     funded|
+-----------+



# - Filter

In [46]:
df_loans.filter(df_loans.status == 'funded').count()

1510115

In [47]:
df_loans.filter("status == 'refunded'").count()

5336

In [48]:
df_loans.where("status == 'expired'").count()

77642

# - Select distinct sector names

In [49]:
df_loans.select('sector_name').distinct().show(truncate = False)

+--------------+
|sector_name   |
+--------------+
|Services      |
|Wholesale     |
|Education     |
|Food          |
|Entertainment |
|Housing       |
|Health        |
|Clothing      |
|Personal Use  |
|Construction  |
|Agriculture   |
|Arts          |
|Retail        |
|Manufacturing |
|Transportation|
+--------------+



# - Number of activities in Services sector

In [51]:
df_loans.where("sector_name = 'Services'").select('activity_name').distinct().show()

+-------------------+
|      activity_name|
+-------------------+
|           Services|
|   Machinery Rental|
|Mobile Transactions|
|  Motorcycle Repair|
|          Computers|
|             Energy|
|          Tailoring|
|      Internet Cafe|
|            Laundry|
|        Electrician|
|           Printing|
|     Communications|
|        Call Center|
|        Bookbinding|
|            Cobbler|
|          Utilities|
|              Hotel|
| Electronics Repair|
|    Vehicle Repairs|
|   Air Conditioning|
+-------------------+
only showing top 20 rows



In [52]:
df_loans.where("sector_name = 'Services'").select('activity_name').distinct().count()

39

# - Number of records with mobile transactions

In [53]:
# 1st Approach

df_loans.where("sector_name = 'Services' and activity_name = 'Mobile Transactions'").count()

1624

In [55]:
# 2nd Approach

df_loans.where((df_loans.sector_name == 'Services') & (df_loans.activity_name == 'Mobile Transactions')).count()

1624

In [56]:
# 3rd Approach

df_loans.where((f.col('sector_name') == 'Services') & (f.col('activity_name') == 'Mobile Transactions')).count()

1624

# - Number of records with Beauty Salon and Barber Shop

In [57]:
df_loans.where((f.col('sector_name') == 'Services') & (f.col('activity_name').isin('Beauty Salon','Barber Shop'))).count()

19558

# - Loans with disburse time between 2015-01-01 and 2018-01-01

In [59]:
df_loans.select('disburse_time').show(10,False)

+-----------------------------+
|disburse_time                |
+-----------------------------+
|2016-04-18 07:00:00.000 +0000|
|2012-12-14 08:00:00.000 +0000|
|2011-09-13 07:00:00.000 +0000|
|2014-09-20 07:00:00.000 +0000|
|2014-04-10 07:00:00.000 +0000|
|2020-03-04 08:00:00.000 +0000|
|2012-10-20 07:00:00.000 +0000|
|2018-05-31 07:00:00.000 +0000|
|2018-08-23 07:00:00.000 +0000|
|2013-04-08 07:00:00.000 +0000|
+-----------------------------+
only showing top 10 rows



# Right format = yyyy-MM-dd
## Wrong format = dd/mm/yyyy

In [64]:
df_loans.select(f.col('disburse_time').substr(1,10).alias('disbursed_date').cast('date')).show()

+--------------+
|disbursed_date|
+--------------+
|    2016-04-18|
|    2012-12-14|
|    2011-09-13|
|    2014-09-20|
|    2014-04-10|
|    2020-03-04|
|    2012-10-20|
|    2018-05-31|
|    2018-08-23|
|    2013-04-08|
|    2017-09-14|
|    2013-12-19|
|    2014-12-19|
|    2012-10-16|
|    2019-04-30|
|    2014-02-20|
|    2019-12-30|
|    2013-06-20|
|    2018-12-26|
|    2014-11-28|
+--------------+
only showing top 20 rows



In [65]:
df_loans.filter(f.col('disburse_time').substr(1,10).cast('date').between('2015-01-01','2018-01-01'))\
.select('loan_id','loan_name','disburse_time').show(truncate = False)

+-------+---------------+-----------------------------+
|loan_id|loan_name      |disburse_time                |
+-------+---------------+-----------------------------+
|1057058|\tLucy         |2016-04-18 07:00:00.000 +0000|
|1383778|Yesica Marilú  |2017-09-14 07:00:00.000 +0000|
|835424 |Vilma          |2015-01-26 08:00:00.000 +0000|
|1296242|Mulyati S      |2017-05-08 07:00:00.000 +0000|
|1233188|Nida           |2017-01-20 08:00:00.000 +0000|
|950357 |Bernaditha     |2015-08-25 07:00:00.000 +0000|
|1272007|Rebecca        |2017-03-06 08:00:00.000 +0000|
|1316761|Glenda         |2017-05-24 07:00:00.000 +0000|
|1124820|ROSALINA       |2016-07-14 07:00:00.000 +0000|
|1306337|Nancy          |2017-05-05 07:00:00.000 +0000|
|1033537|Susan          |2016-02-05 08:00:00.000 +0000|
|871235 |Durdona        |2015-03-17 07:00:00.000 +0000|
|987778 |Thanh          |2015-10-31 07:00:00.000 +0000|
|938334 |Rafael de Jesus|2015-08-11 07:00:00.000 +0000|
|1421780|Leonara        |2017-10-27 07:00:00.000